# Nearest Neighbors

## Import module

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

from NN_func import load_sparse_csr
from NN_func import word_count_by_name
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances

## Load Wikipedia dataset

In [2]:
# Load Data
wiki = pd.read_csv("people_wiki.csv")
wiki['length'] = wiki.apply(lambda x: len(x['text']),axis=1)

Explore some data:

In [3]:
wiki.head(5)

,URI,name,text,length
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,1479
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,1466
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,1297
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,2364
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,1167


## Extract word count vectors

To load in the word count vectors, define the function:

In [4]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

In [ ]:
# word count vector
word_count = load_sparse_csr('people_wiki_word_count.npz')

The word-to-index mapping is given by:

In [ ]:
idx_to_word = pd.read_json("people_wiki_map_index_to_word.json", typ='series')
idx_to_word = idx_to_word.sort_values()

Let's see some of indices:

In [ ]:
idx_to_word

## Find nearest neighbors

First, let's define a functino to find word count easily by name:

In [ ]:
def word_count_by_name(data, count_vector, name):
    wiki_id = data[data['name']==name].index[0]
    return count_vector[wiki_id]

In [ ]:
model = NearestNeighbors(metric='euclidean', algorithm = 'brute')
model.fit(word_count)

In [ ]:
# row number of Obama's article
wiki[wiki['name']=='Barack Obama']

10 nearest neighbors of Barack Obama:

In [ ]:
# row number of Obama's article
distances, indices = model.kneighbors(word_count_by_name(wiki,word_count,'Barack Obama'), n_neighbors=10)
nn_obama = wiki.loc[indices[0]]
nn_obama['distance'] = distances[0]
nn_obama[['name','distance']]

In [ ]:
# Barack Obama word count table
obama_words = pd.DataFrame(word_count_by_name(wiki,word_count,'Barack Obama').toarray()[0],index=idx_to_word.index,columns=['count'])
obama_words = obama_words.sort_values(by='count',ascending=False)
print("Obama words: \n", obama_words.head(10))

All of the 10 people are politicians, but about half of them have rather tenuous connections with Obama, other than the fact that they are politicians.

* Francisco Barrio is a Mexican politician, and a former governor of Chihuahua.
* Walter Mondale and Don Bonker are Democrats who made their career in late 1970s.
* Wynn Normington Hugh-Jones is a former British diplomat and Liberal Party official.
* Andy Anstett is a former politician in Manitoba, Canada.

Nearest neighbors with raw word counts got some things right, showing all politicians in the query result, but missed finer and important details.

For instance, let's find out why Francisco Barrio was considered a close neighbor of Obama. To do this, let's look at the most frequently used words in each of Barack Obama and Francisco Barrio's pages:

In [ ]:
# row number of Francisco Barrio's article
wiki[wiki['name']=='Francisco Barrio']

In [ ]:
distances, indices = model.kneighbors(word_count_by_name(wiki,word_count,'Francisco Barrio'), n_neighbors=10)
nn_barrio = wiki.loc[indices[0]]
nn_barrio['distance'] = distances[0]
nn_barrio[['name','distance']]

In [ ]:
barrio_words = pd.DataFrame(word_count_by_name(wiki,word_count,'Francisco Barrio').toarray()[0],index=idx_to_word.index,columns=['count'])
barrio_words = barrio_words.sort_values(by='count',ascending=False)
print("Barrio words: \n", barrio_words.head(10))

Let's extract the list of most frequent words that appear in both Obama's and Barrio's documents. We've so far sorted all words from Obama and Barrio's articles by their word frequencies. We will now use a dataframe operation known as join. The join operation is very useful when it comes to playing around with data: it lets you combine the content of two tables using a shared column (in this case, the word column).

In [ ]:
# Combined Word
combined_words = obama_words.join(barrio_words,rsuffix='.1')
# Rename Columns
combined_words = combined_words.rename(columns={'count':'Obama','count.1':'Barrio'})
# Sort by obama
combined_words = combined_words.sort_values(by='Obama', ascending = False)
combined_words

## Distances

In [ ]:
print("Distance between Obama and George W. Bush: \n", euclidean_distances(word_count_by_name(wiki,word_count,'Barack Obama'),word_count_by_name(wiki,word_count,'George W. Bush'))[0])
print("Distance between Obama and Joe Biden: \n", euclidean_distances(word_count_by_name(wiki,word_count,'Barack Obama'),word_count_by_name(wiki,word_count,'Joe Biden'))[0])
print("Distance between George W. Bush and Joe Biden: \n", euclidean_distances(word_count_by_name(wiki,word_count,'Joe Biden'),word_count_by_name(wiki,word_count,'George W. Bush'))[0])

## TF-IDF to the rescue

Much of the perceived commonalities between Obama and Barrio were due to occurrences of extremely frequent words, such as "the", "and", and "his". So nearest neighbors is recommending plausible results sometimes for the wrong reasons.

To retrieve articles that are more relevant, we should focus more on rare words that don't happen in every article. **TF-IDF(term frequency–inverse document frequency)** is a feature representation that penalizes words that are too common. Let's use GraphLab Create's implementation of TF-IDF and repeat the search for the 10 nearest neighbors of Barack Obama:

In [ ]:
tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')

In [ ]:
# NN model
model = NearestNeighbors(metric='euclidean', algorithm = 'brute')
model.fit(tf_idf)

In [ ]:
distances, indices = model.kneighbors(word_count_by_name(wiki,tf_idf,'Barack Obama'), n_neighbors=100)
nn_obama = wiki.loc[indices[0]]
nn_obama['distance'] = distances[0]
nn_obama[['name','distance']]

Clearly, the results are more plausible with the use of TF-IDF. Let's take a look at the word vector for Obama and Schilirio's pages. Notice that TF-IDF representation assigns a weight to each word. This weight captures relative importance of that word in the document. Let us sort the words in Obama's article by their TF-IDF weights; we do the same for Schiliro's article as well.

In [ ]:
# Barack Obama word count table
obama_words = pd.DataFrame(word_count_by_name(wiki,tf_idf,'Barack Obama').toarray()[0],index=idx_to_word.index,columns=['weight'])
obama_words = obama_words.sort_values(by='weight',ascending=False)
obama_words.head(10)

In [ ]:
# Schiliro Obama word count table
schiliro_words = pd.DataFrame(word_count_by_name(wiki,tf_idf,'Phil Schiliro').toarray()[0],index=idx_to_word.index,columns=['weight'])
schiliro_words = schiliro_words.sort_values(by='weight',ascending=False)
schiliro_words.head(10)

Let's combine these tables:

In [ ]:
# combined words
combined_words = obama_words.join(schiliro_words,rsuffix='.1')
# Rename Columns
combined_words = combined_words.rename(columns={'weight':'Obama','weight.1':'Barrio'})
# Sort by obama
combined_words = combined_words.sort_values(by='Obama', ascending = False)
combined_words

## Distances

In [ ]:
# Euclidean Distance
# Obama and Bush
print("Distance between Obama and George W. Bush: \n", euclidean_distances(word_count_by_name(wiki,tf_idf,'Barack Obama'),word_count_by_name(wiki,tf_idf,'George W. Bush'))[0])
print("Distance between Obama and Joe Biden: \n", euclidean_distances(word_count_by_name(wiki,tf_idf,'Barack Obama'),word_count_by_name(wiki,tf_idf,'Joe Biden'))[0])
print("Distance between George W. Bush and Joe Biden: \n", euclidean_distances(word_count_by_name(wiki,tf_idf,'Joe Biden'),word_count_by_name(wiki,tf_idf,'George W. Bush'))[0])

## Choosing metrics

In [ ]:
# length of document
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nn_obama['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.axvline(x=wiki[wiki['name']=='Barack Obama']['length'].values[0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki[wiki['name']=='Joe Biden']['length'].values[0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([1000, 5500, 0, 0.004])

plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size':16})
plt.tight_layout()

Relative to the rest of Wikipedia, nearest neighbors of Obama are overwhemingly short, most of them being shorter than 2000 words. The bias towards short articles is not appropriate in this application as there is really no reason to favor short articles over long articles (they are all Wikipedia articles, after all). Many Wikipedia articles are 2500 words or more, and both Obama and Biden are over 2500 words long.

**Note**: Both word-count features and TF-IDF are proportional to word frequencies. While TF-IDF penalizes very common words, longer articles tend to have longer TF-IDF vectors simply because they have more words in them.

To remove this bias, we turn to **cosine distances**: 

$$
d(\mathbf{x},\mathbf{y}) = 1 - \frac{\mathbf{x}^T\mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|}
$$ 

Cosine distances let us compare word distributions of two articles of varying lengths.

Let us train a new nearest neighbor model, this time with cosine distances. We then repeat the search for Obama's 100 nearest neighbors.

In [ ]:
# cosine distance model
model_cosine= NearestNeighbors(metric='cosine', algorithm = 'brute')
model_cosine.fit(tf_idf)

In [ ]:
distances, indices = model_cosine.kneighbors(word_count_by_name(wiki,tf_idf,'Barack Obama'), n_neighbors=100)
nn_obama_cosine = wiki.loc[indices[0]]
nn_obama_cosine['distance'] = distances[0]
nn_obama_cosine[['name','distance','length']]

From a glance at the above table, things look better. For example, we now see Joe Biden as Barack Obama's nearest neighbor! We also see Hillary Clinton on the list. This list looks even more plausible as nearest neighbors of Barack Obama.

Let's make a plot to better visualize the effect of having used cosine distance in place of Euclidean on our TF-IDF vectors.

In [ ]:
# with cosine distance
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nn_obama['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.hist(nn_obama_cosine['length'], 50, color='b', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (cosine)', zorder=11, alpha=0.8)
plt.axvline(x=wiki[wiki['name']=='Barack Obama']['length'].values[0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki[wiki['name']=='Joe Biden']['length'].values[0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([1000, 5500, 0, 0.004])
plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size': 16})
plt.tight_layout()

## Problem with cosine distances: tweets vs. long articles

Happily ever after? Not so fast. Cosine distances ignore all document lengths, which may be great in certain situations but not in others. For instance, consider the following (admittedly contrived) example.

```
+--------------------------------------------------------+
|                                             +--------+ |
|  One that shall not be named                | Follow | |
|  @username                                  +--------+ |
|                                                        |
|  Democratic governments control law in response to     |
|  popular act.                                          |
|                                                        |
|  8:05 AM - 16 May 2016                                 |
|                                                        |
|  Reply   Retweet (1,332)   Like (300)                  |
|                                                        |
+--------------------------------------------------------+
```
How similar is this tweet to Barack Obama's Wikipedia article? Let's transform the tweet into TF-IDF features, using an encoder fit to the Wikipedia dataset. (That is, let's treat this tweet as an article in our Wikipedia dataset and see what happens.)

In [ ]:
# Problem with cosine distances: tweets vs. long articles
tweet = {'act': 3.4597778278724887,
 'control': 3.721765211295327,
 'democratic': 3.1026721743330414,
 'governments': 4.167571323949673,
 'in': 0.0009654063501214492,
 'law': 2.4538226269605703,
 'popular': 2.764478952022998,
 'response': 4.261461747058352,
 'to': 0.04694493768179923}

In [ ]:
word_indices = [idx_to_word[idx_to_word.index==word].values[0] for word in tweet.keys()]
tweet_tf_idf = csr_matrix((list(tweet.values()),([0]*len(word_indices), word_indices)),
                          shape=(1, tf_idf.shape[1]) )

In [ ]:
cosine_distances(word_count_by_name(wiki,tf_idf,'Barack Obama'),tweet_tf_idf)

With cosine distances, the tweet is "nearer" to Barack Obama than everyone else, except for Joe Biden! This probably is not something we want. If someone is reading the Barack Obama Wikipedia page, would you want to recommend they read this tweet? Ignoring article lengths completely resulted in nonsensical results. In practice, it is common to enforce maximum or minimum document lengths. After all, when someone is reading a long article from The Atlantic, you wouldn't recommend him/her a tweet.